In [ ]:
# ============================================================
# Stock Price Big Data Analysis với PySpark
# Phân tích dữ liệu chứng khoán từ HDFS
# ============================================================

# Cell 1: Import thư viện và khởi tạo Spark Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Tạo Spark Session kết nối với Spark Master
spark = SparkSession.builder \
    .appName("Stock Price Big Data Analysis") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print("=" * 60)
print("✅ Spark Session khởi tạo thành công!")
print(f"📌 Spark Version: {spark.version}")
print(f"📌 Spark Master: {spark.sparkContext.master}")
print(f"📌 Application Name: {spark.sparkContext.appName}")
print("=" * 60)

# ============================================================
# Cell 2: Đọc dữ liệu từ HDFS
print("\n📂 Đang đọc dữ liệu từ HDFS...")

# Định nghĩa schema cho dữ liệu stock
schema = StructType([
    StructField("Date", DateType(), True),
    StructField("Open", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Low", DoubleType(), True),
    StructField("Close", DoubleType(), True),
    StructField("Volume", LongType(), True)
])

# Đọc tất cả file CSV từ HDFS (path: hdfs://namenode:9000/datack/)
# Lưu ý: Đường dẫn này trùng với cấu trúc trong README
df_stock = spark.read \
    .option("header", "true") \
    .option("inferSchema", "false") \
    .schema(schema) \
    .csv("hdfs://namenode:9000/datack/*.csv")

# Thêm cột ticker từ tên file
df_stock = df_stock.withColumn(
    "Ticker",
    regexp_extract(input_file_name(), r"stock_market_data-([A-Z]+)_", 1)
)

print(f"✅ Đã đọc dữ liệu từ HDFS thành công!")
print(f"📊 Tổng số records: {df_stock.count():,}")
print(f"📊 Số lượng cột: {len(df_stock.columns)}")
print(f"📊 Số lượng công ty: {df_stock.select('Ticker').distinct().count()}")

# Hiển thị schema
print("\n📋 Schema của dữ liệu:")
df_stock.printSchema()

# Hiển thị 10 dòng đầu tiên
print("\n📋 10 dòng dữ liệu đầu tiên:")
df_stock.show(10, truncate=False)

# ============================================================
# Cell 3: Làm sạch và xử lý dữ liệu
print("\n🔧 Bắt đầu làm sạch dữ liệu...")

# Loại bỏ các record có giá trị null
df_clean = df_stock.dropna()

# Tính các chỉ số kỹ thuật
df_clean = df_clean.withColumn("Price_Range", col("High") - col("Low"))
df_clean = df_clean.withColumn("Daily_Return", (col("Close") - col("Open")) / col("Open") * 100)
df_clean = df_clean.withColumn("Year", year(col("Date")))
df_clean = df_clean.withColumn("Month", month(col("Date")))

# Cache dữ liệu để tăng tốc độ xử lý
df_clean.cache()

print(f"✅ Làm sạch dữ liệu hoàn tất!")
print(f"📊 Số records sau khi làm sạch: {df_clean.count():,}")

# Hiển thị dữ liệu sau khi xử lý
print("\n📋 Dữ liệu sau khi xử lý:")
df_clean.select("Ticker", "Date", "Close", "Daily_Return", "Price_Range").show(10)

# ============================================================
# Cell 4: Phân tích thống kê cơ bản
print("\n📊 PHÂN TÍCH THỐNG KÊ CƠ BẢN")
print("=" * 60)

# Thống kê mô tả
print("\n1️⃣ Thống kê mô tả cho các cột giá:")
df_clean.select("Open", "High", "Low", "Close", "Volume").describe().show()

# Top 10 công ty có khối lượng giao dịch cao nhất
print("\n2️⃣ Top 10 công ty có khối lượng giao dịch trung bình cao nhất:")
top_volume = df_clean.groupBy("Ticker") \
    .agg(avg("Volume").alias("Avg_Volume")) \
    .orderBy(desc("Avg_Volume")) \
    .limit(10)
top_volume.show()

# Top 10 công ty có giá đóng cửa trung bình cao nhất
print("\n3️⃣ Top 10 công ty có giá đóng cửa trung bình cao nhất:")
top_price = df_clean.groupBy("Ticker") \
    .agg(avg("Close").alias("Avg_Close_Price")) \
    .orderBy(desc("Avg_Close_Price")) \
    .limit(10)
top_price.show()

# Top 10 công ty có biến động giá cao nhất
print("\n4️⃣ Top 10 công ty có biến động giá cao nhất:")
top_volatility = df_clean.groupBy("Ticker") \
    .agg(avg("Price_Range").alias("Avg_Price_Range")) \
    .orderBy(desc("Avg_Price_Range")) \
    .limit(10)
top_volatility.show()

# ============================================================
# Cell 5: Phân tích xu hướng theo thời gian
print("\n📈 PHÂN TÍCH XU HƯỚNG THEO THỜI GIAN")
print("=" * 60)

# Thống kê theo năm
print("\n1️⃣ Giá trung bình và khối lượng giao dịch theo năm:")
yearly_stats = df_clean.groupBy("Year") \
    .agg(
        avg("Close").alias("Avg_Close"),
        avg("Volume").alias("Avg_Volume"),
        count("*").alias("Total_Records")
    ) \
    .orderBy("Year")
yearly_stats.show()

# Phân tích theo tháng
print("\n2️⃣ Giá trung bình theo tháng (tất cả các năm):")
monthly_stats = df_clean.groupBy("Month") \
    .agg(
        avg("Close").alias("Avg_Close"),
        avg("Daily_Return").alias("Avg_Return")
    ) \
    .orderBy("Month")
monthly_stats.show()

# ============================================================
# Cell 6: Phân tích các cổ phiếu cụ thể (AAPL, GOOGL, AMZN, MSFT)
print("\n🎯 PHÂN TÍCH CÁC CỔ PHIẾU QUAN TRỌNG")
print("=" * 60)

# Lọc các cổ phiếu quan trọng
major_stocks = ["AAPL", "GOOGL", "AMZN", "MSFT", "TSLA"]
df_major = df_clean.filter(col("Ticker").isin(major_stocks))

print(f"\n📊 Phân tích {len(major_stocks)} cổ phiếu: {', '.join(major_stocks)}")

# Thống kê cho từng cổ phiếu
for ticker in major_stocks:
    df_ticker = df_major.filter(col("Ticker") == ticker)
    
    stats = df_ticker.select(
        min("Close").alias("Min_Price"),
        max("Close").alias("Max_Price"),
        avg("Close").alias("Avg_Price"),
        stddev("Close").alias("Std_Price")
    ).collect()[0]
    
    print(f"\n{ticker}:")
    print(f"  - Giá thấp nhất: ${stats['Min_Price']:.2f}")
    print(f"  - Giá cao nhất: ${stats['Max_Price']:.2f}")
    print(f"  - Giá trung bình: ${stats['Avg_Price']:.2f}")
    print(f"  - Độ lệch chuẩn: ${stats['Std_Price']:.2f}")

# ============================================================
# Cell 7: Phân tích tương quan
print("\n🔗 PHÂN TÍCH TƯƠNG QUAN")
print("=" * 60)

# Tính tương quan giữa các biến
print("\n1️⃣ Ma trận tương quan giữa các biến giá:")
correlation_cols = ["Open", "High", "Low", "Close", "Volume"]

# Chuyển sang Pandas DataFrame để dễ tính correlation
df_sample = df_clean.select(correlation_cols).sample(fraction=0.1).toPandas()

correlation_matrix = df_sample.corr()
print("\n", correlation_matrix)

# ============================================================
# Cell 8: Visualization với Matplotlib
print("\n📊 TẠO VISUALIZATION")
print("=" * 60)

# Convert một phần dữ liệu sang Pandas để visualization
df_viz = df_clean.filter(col("Ticker").isin(["AAPL", "GOOGL", "MSFT"])) \
    .filter(col("Year") >= 2018) \
    .select("Date", "Ticker", "Close") \
    .toPandas()

# Sắp xếp theo ngày
df_viz = df_viz.sort_values("Date")

# Plot 1: Line chart so sánh giá cổ phiếu
plt.figure(figsize=(14, 6))
for ticker in ["AAPL", "GOOGL", "MSFT"]:
    data = df_viz[df_viz["Ticker"] == ticker]
    plt.plot(data["Date"], data["Close"], label=ticker, linewidth=2)

plt.title("So sánh giá cổ phiếu AAPL, GOOGL, MSFT (2018-2020)", fontsize=14, fontweight='bold')
plt.xlabel("Ngày", fontsize=12)
plt.ylabel("Giá đóng cửa ($)", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("/home/jovyan/work/stock_comparison.png", dpi=300)
print("✅ Đã lưu biểu đồ: stock_comparison.png")
plt.show()

# Plot 2: Volume distribution
df_volume = top_volume.toPandas()
plt.figure(figsize=(12, 6))
plt.barh(df_volume["Ticker"], df_volume["Avg_Volume"], color='steelblue')
plt.xlabel("Khối lượng giao dịch trung bình", fontsize=12)
plt.title("Top 10 công ty có khối lượng giao dịch cao nhất", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("/home/jovyan/work/top_volume.png", dpi=300)
print("✅ Đã lưu biểu đồ: top_volume.png")
plt.show()

# Plot 3: Heatmap tương quan
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title("Ma trận tương quan giữa các biến", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("/home/jovyan/work/correlation_heatmap.png", dpi=300)
print("✅ Đã lưu biểu đồ: correlation_heatmap.png")
plt.show()

# ============================================================
# Cell 9: Lưu kết quả phân tích vào HDFS
print("\n💾 LƯU KẾT QUẢ PHÂN TÍCH VÀO HDFS")
print("=" * 60)

# Lưu top companies theo volume
print("\n1️⃣ Lưu top companies by volume...")
top_volume.write.mode("overwrite").parquet("hdfs://namenode:9000/output/top_volume")
print("✅ Đã lưu vào: hdfs://namenode:9000/output/top_volume")

# Lưu yearly statistics
print("\n2️⃣ Lưu yearly statistics...")
yearly_stats.write.mode("overwrite").parquet("hdfs://namenode:9000/output/yearly_stats")
print("✅ Đã lưu vào: hdfs://namenode:9000/output/yearly_stats")

# Lưu processed data
print("\n3️⃣ Lưu processed data sample...")
df_clean.filter(col("Year") == 2020).write.mode("overwrite") \
    .parquet("hdfs://namenode:9000/output/processed_2020")
print("✅ Đã lưu vào: hdfs://namenode:9000/output/processed_2020")

# ============================================================
# Cell 10: Tổng kết và dọn dẹp
print("\n" + "=" * 60)
print("✅ HOÀN THÀNH PHÂN TÍCH DỮ LIỆU CHỨNG KHOÁN")
print("=" * 60)

print("\n📊 Tóm tắt kết quả:")
print(f"  • Tổng số records đã xử lý: {df_clean.count():,}")
print(f"  • Số lượng công ty: {df_clean.select('Ticker').distinct().count()}")
print(f"  • Khoảng thời gian: {df_clean.select(min('Date')).collect()[0][0]} - {df_clean.select(max('Date')).collect()[0][0]}")
print(f"  • Số biểu đồ đã tạo: 3")
print(f"  • Số file output trên HDFS: 3")

print("\n📁 Các file đã tạo:")
print("  • /home/jovyan/work/stock_comparison.png")
print("  • /home/jovyan/work/top_volume.png")
print("  • /home/jovyan/work/correlation_heatmap.png")

print("\n📁 Dữ liệu đã lưu trên HDFS:")
print("  • hdfs://namenode:9000/output/top_volume")
print("  • hdfs://namenode:9000/output/yearly_stats")
print("  • hdfs://namenode:9000/output/processed_2020")

# Dừng Spark Session
# spark.stop()
print("\n🎉 Hoàn tất! Spark Session vẫn đang chạy.")